In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
from gw_eccentricity import measure_eccentricity, load_waveform, load_data
from gw_eccentricity.utils import interpolate
from pycbc.waveform import waveform_modes, get_td_waveform
import h5py
import os
import pylab as plt
import numpy as np
from lal import MTSUN_SI, PC_SI, C_SI
from pycbc import frame
import pycbc.conversions as convert
from scipy.interpolate import interp1d
import pandas as pd

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
gwe_defaults = load_data.get_load_waveform_defaults('SXSCatalog')
gwe_defaults

{'data_dir': None,
 'metadata_path': None,
 'deltaTOverM': 0.1,
 'include_zero_ecc': False,
 'include_params_dict': False,
 'zero_ecc_approximant': 'IMRPhenomT',
 'num_orbits_to_remove_as_junk': 2,
 'mode_array': [(2, 2)],
 'extrap_order': 2}

In [3]:
def x_from_f(f, total_mass):
    """
    Parameters:
    ----------------------------------
    f: float
       GW frequency (in Hz)
    total_mass: float
                Total mass of the system (in solar masses)
                
    ------------------------------------------------------------
    Returns: dimensionless frequency corresponding to f for the given total_mass
    """
    return (np.pi * total_mass * MTSUN_SI * f)**(2/3)

def M_by_R(total_mass, dist):
    """
    Parameters:
    total_mass: total mass (M) in M_Sun
    dist: Luminosity distance (R) in Mpc
    Returns M/R in dimensionless units
    """
    dist_in_m = dist*PC_SI*10**6  #distance in meters
    mass_in_m = total_mass*MTSUN_SI*C_SI  #total mass in meters
    return(mass_in_m/dist_in_m)

In [4]:
with h5py.File('/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/hybrid_data/EccTD_Ebersold_hybrids/modified_hybrids/1364_EccTD_Ebersold_HM_modified.h5', 'r') as f:
    print(f['l2_m2'])

<HDF5 dataset "l2_m2": shape (26188, 3), type "<f8">


In [2]:
measure_eccentricity?

Signature:
measure_eccentricity(
    tref_in=None,
    fref_in=None,
    method='Amplitude',
    dataDict=None,
    num_orbits_to_exclude_before_merger=2,
    extra_kwargs=None,
)
Docstring:
Measure eccentricity and mean anomaly from a gravitational waveform.

Eccentricity is measured using the GW frequency omega22(t) =
dphi22(t)/dt, where phi22(t) is the phase of the (2, 2) waveform
mode. We currently only allow time-domain, nonprecessing waveforms. We
evaluate omega22(t) at pericenter times, t_pericenters, and build a
spline interpolant omega22_pericenters(t) using those data
points. Similarly, we build omega22_apocenters(t) using omega22(t) at
the apocenter times, t_apocenters.

Using omega22_pericenters(t) and omega22_apocenters(t), we first
compute e_omega22(t), as described in Eq.(4) of arXiv:2302.11257. We
then use e_omega22(t) to compute the eccentricity egw(t) using Eq.(8)
of arXiv:2302.11257. Mean anomaly is defined using t_pericenters, as
described in Eq.(10) of arXiv:2302.1

# Hybrids

In [52]:
wf_files_path = '/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/hybrid_data/EccTD_Ebersold_hybrids/modified_hybrids/hybrid_wfs_data_22_mode_modified.h5'

In [53]:
mode = [2, 2]
dataDict = {}
M = 35
iota = np.pi/6
DL = 1

for nr_approx in ['BBH_1364']:
    dataDict[nr_approx] = {}
    with h5py.File(wf_files_path, 'r') as data_from_file:
        approx_data = data_from_file[nr_approx]
        t_by_M = approx_data['t_by_M'][:]
        Re_rh_by_M = approx_data['Real_rh_by_M'][:]
        Im_rh_by_M = approx_data['Imag_rh_by_M'][:]

        #t_peak_idx = np.argmax(np.abs(Re_rh_by_M - 1j*Im_rh_by_M))        
        #t_by_M_shifted = (t_by_M - t_by_M[t_peak_idx])

        t_sec = t_by_M*M*MTSUN_SI
        t_interp = np.arange(t_by_M[0], t_by_M[-1], gwe_defaults["deltaTOverM"])
        t_interp_sec = t_interp*M*MTSUN_SI  #Multiply by total mass in seconds

        Y_lm = waveform_modes.get_glm(mode[0], mode[1], iota)  #taking phi_0 = 0
        hp = Re_rh_by_M * M_by_R(M, DL) * Y_lm
        hc = -Im_rh_by_M * M_by_R(M, DL) * Y_lm

        hp_interp = interpolate(t_interp_sec, t_sec, hp)
        hc_interp = interpolate(t_interp_sec, t_sec, hc)
        hlm_interp = hp_interp - 1j * hc_interp

        dataDict[nr_approx]['t'] = t_interp_sec
        dataDict[nr_approx]['hlm'] = {(2,2):hlm_interp}
        
        print(measure_eccentricity(fref_in=20, dataDict=dataDict[nr_approx]))

{'eccentricity': np.float64(-0.19060009495186936), 'mean_anomaly': np.float64(6.160351912998841), 'fref_out': np.int64(20), 'gwecc_object': <gw_eccentricity.eccDefinitionUsingAmplitude.eccDefinitionUsingAmplitude object at 0x7814fbb65d20>}


/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1461: UserWarning: Encountered negative eccentricity.
For more verbose output use `debug_level=1`.
  debug_message("Encountered negative eccentricity.",
/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1788: UserWarning: egw(t) is nonmonotonic in the range [np.float64(-2.7309450358828236), np.float64(0.2774364741465587)]
For more verbose output use `debug_level=1`.
  debug_message(message, self.debug_level,


# TEOBResumS

In [55]:
M = 35
pi = np.pi
DL = 410
iota = pi/6
chi_1z = 0.3
chi_2z = 0.3
srate = 4096

for q in [1.25, 2, 3]:
    print(f'q = {q}')
    m1 = convert.mass1_from_mtotal_q(M, q)
    m2 = convert.mass2_from_mtotal_q(M, q)
    for ecc in np.arange(0.0, 0.35, 0.05):
        hp, hc = get_td_waveform(approximant='teobresums', mass1=m1, mass2=m2,
                                 f_lower=18, delta_t=1.0/srate, phase=0, distance = DL, 
                                 lambda1=0, lambda2=0, spin1z=chi_1z, spin2z=chi_2z, 
                                 eccentricity=ecc)
        hlm = hp + 1j*hc
        
        #dataDict_teob = {"t": t_teob, "hlm": {(2, 2): h22_ecc_teob}}
        dataDict_teob = {'t':hp.sample_times.data, 'hlm':{(2,2):hlm.data}}
        result_dict = measure_eccentricity(fref_in = 20, dataDict = dataDict_teob)
        print(round(result_dict['eccentricity'], 2))
    print('\n')

q = 1.25
0.0
0.05
0.09
0.14
0.19
0.23
0.29


q = 2
0.0
0.05
0.09
0.14
0.19
0.23
0.29


q = 3
0.0
0.05
0.09
0.14
0.19
0.23
0.29




# ICTS Sims

In [5]:
pycbc_to_metadata_dict = {
    'chi_eff': 'chi_eff_ref',
    'chi_p': 'chi_p_ref',
    'distance': 'DL',
    'mass1': 'm1',
    'mass2': 'm2',
    'mchirp': 'm_chirp',
    'mtotal': 'M',
    'polarization': 'pol',
    'eccentricity': 'Eccentricity',
    'metadata_ecc': 'Eccentricity'
}

keys_not_in_metadata = {
    'coa_phase': 0, 
    'delta_tc': 0,
    'loglikelihood': None,
    'logwt': None
}

In [6]:
def get_param_inj_val_from_metadata(meta_file, param, round_to=4):
    with open(meta_file, 'r') as f:
        metadata_text = f.read().splitlines()

    while(1):
        try:
            metadata_text.remove('')
        except ValueError:
            break
    
    metadata_text = [line.strip().replace('\\t', '') for line in metadata_text]
    for line in metadata_text:
        if '=' not in line or '===>' in line:
            continue
        param_metadata, val = ''.join(line.split()).split('=')
        return_val = val
        if param == 'spin1z' and param_metadata == 'ChiA_ref':
            return_val = eval(val)[-1]
            break
        elif param == 'spin2z' and param_metadata == 'ChiB_ref':
            return_val = eval(val)[-1]
            break
        elif param_metadata == param:
            break
        elif param in pycbc_to_metadata_dict.keys():
            if param_metadata == pycbc_to_metadata_dict[param]:
                break
    else:
        raise ValueError(f'{param} value not found')
    return(round(float(return_val), round_to))

In [7]:
base_dir = '/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/eccentric_pe/'
run_dir_base = os.path.join(base_dir, 'injections/ICTS_sims')
ecc_spin_run_nums = {'PS': 'run01',
                     'eAS': 'run02'}

In [15]:
sims_at_highest_lev = {'names':[], 'meta_files':[], 'levs':[]}
for lev in ['Lev4', 'Lev3', 'Lev2']:
    sim_list = list(os.walk(os.path.join(run_dir_base, lev)))[0][1]
    sim_list.sort()
    for sim in sim_list:
        if sim not in sims_at_highest_lev['names']:
            sims_at_highest_lev['names'].append(sim)
            sims_at_highest_lev['meta_files'].append(os.path.join(run_dir_base, 
                                                                  lev, 
                                                                  sim, 
                                                                  'zero_noise',
                                                                  'inj1',
                                                                  'metadata.txt'))
            sims_at_highest_lev['levs'].append(lev)
sims_at_highest_lev_df = pd.DataFrame(sims_at_highest_lev).set_index('names')

In [16]:
inj_vals = {
    'sim_names': [],
    'mchirp': [],
    'chi_eff': [],
    'chi_p': [],
    'mtotal': [],
    'q': [],
    'metadata_ecc': []
}
for sim, meta_file in zip(sims_at_highest_lev['names'], sims_at_highest_lev['meta_files']):
    inj_vals['sim_names'].append(sim)
    for param in list(inj_vals.keys())[1:]:
        inj_vals[param].append(get_param_inj_val_from_metadata(meta_file, param))
inj_vals['gwecc_ecc'] = [np.nan]*len(inj_vals['mtotal'])
inj_vals['gwecc_mean_anomaly'] = [np.nan]*len(inj_vals['mtotal'])
inj_vals_df = pd.DataFrame(inj_vals).set_index('sim_names')

In [26]:
temp_inj_vals_df = pd.DataFrame({key:inj_vals[key] for key in ['sim_names', 'mtotal']})
temp_inj_vals_df.to_csv('ICTSSims_lowest_M.csv')

In [17]:
sims_spin = {'AS':[], 'PS':[]}
for sim in inj_vals_df.index:
    if round(inj_vals_df['chi_p'][sim], 2) == 0:
        sims_spin['AS'].append(sim)
    else:
        sims_spin['PS'].append(sim)

## Aligned-spins

In [18]:
sims_spin['AS']

['ICTSEccParallel04',
 'ICTSEccParallel05',
 'ICTSEccParallel07',
 'ICTSEccParallel08',
 'ICTSEccParallel10',
 'ICTSEccParallel12',
 'ICTSEccParallel17',
 'ICTSEccParallel13',
 'ICTSEccParallel14',
 'ICTSEccParallel15',
 'ICTSEccParallel16']

In [19]:
sim_files_dir = '/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/sims_from_Prayush_Vaishak/ICTSEccParallel'

In [20]:
mode = [2, 2]
dataDict = {}
iota = np.pi/6
DL = 410
unsuccessful_sims = []

for sim in sims_spin['AS']:
    print(sim)
    M = inj_vals_df['mtotal'][sim]
    print('M =', M)
    dataDict[sim] = {}
    wf_files_path = os.path.join(sim_files_dir, 
                                 f'ICTSEccParallel_{sims_at_highest_lev_df["levs"][sim]}_data_{mode[0]}{mode[1]}_mode.h5')
    with h5py.File(wf_files_path, 'r') as data_from_file:
        approx_data = data_from_file[sim]
        t_by_M = approx_data['t_by_M'][:]
        Re_rh_by_M = approx_data['Real_rh_by_M'][:]
        Im_rh_by_M = approx_data['Imag_rh_by_M'][:]

        #t_peak_idx = np.argmax(np.abs(Re_rh_by_M - 1j*Im_rh_by_M))        
        #t_by_M_shifted = (t_by_M - t_by_M[t_peak_idx])

        t_sec = t_by_M*M*MTSUN_SI
        t_interp = np.arange(t_by_M[0], t_by_M[-1], gwe_defaults["deltaTOverM"])
        t_interp_sec = t_interp*M*MTSUN_SI  #Multiply by total mass in seconds

        Y_lm = waveform_modes.get_glm(mode[0], mode[1], iota)  #taking phi_0 = 0
        hp = Re_rh_by_M * M_by_R(M, DL) * Y_lm
        hc = -Im_rh_by_M * M_by_R(M, DL) * Y_lm

        hp_interp = interpolate(t_interp_sec, t_sec, hp)
        hc_interp = interpolate(t_interp_sec, t_sec, hc)
        hlm_interp = hp_interp - 1j * hc_interp

        dataDict[sim]['t'] = t_interp_sec
        dataDict[sim]['hlm'] = {(2,2):hlm_interp}

        try:
            ecc_dict = measure_eccentricity(fref_in=20, dataDict=dataDict[sim])
            print('measured eccentricity on first try')
            inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
            inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']
        except Exception as e:
            try:                
                if 'Reference frequency is outside the allowed range' in str(e):
                    fref_in = np.ceil(float(str(e).split('[')[-1].split(']')[0].split(',')[0]))
                    ecc_dict = measure_eccentricity(fref_in=fref_in, dataDict=dataDict[sim])
                    print(f'measured eccentricity with fref_in = {fref_in}')
                elif 'treat_mid_points_between_pericenters_as_apocenters' in str(e):
                    ecc_dict = measure_eccentricity(fref_in=20, dataDict=dataDict[sim], 
                                                    extra_kwargs={'treat_mid_points_between_pericenters_as_apocenters': True})
                    print('measured eccentricity with "treat_mid_points_between_pericenters_as_apocenters = True" option')
                else:
                    raise Exception
                inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
                inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']
            except:
                unsuccessful_sims.append(sim)
                print('Unable to determine eccentricity. Skipping')
    print('\n')

ICTSEccParallel04
M = 92.0
measured eccentricity with "treat_mid_points_between_pericenters_as_apocenters = True" option


ICTSEccParallel05
M = 110.0
Unable to determine eccentricity. Skipping


ICTSEccParallel07
M = 100.0
measured eccentricity on first try


ICTSEccParallel08
M = 90.0
measured eccentricity with fref_in = 21.0


ICTSEccParallel10
M = 49.0


/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/utils.py:285: UserWarning: No of data points is 3 but spline order k = 3. Decreasing k to 2.
  warnings.warn(f"No of data points is {len(oldX)} but "
/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1788: UserWarning: egw(t) is nonmonotonic in the range [np.float64(0.08726424060556583), np.float64(0.17734949984154041)]
For more verbose output use `debug_level=1`.
  debug_message(message, self.debug_level,
/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1788: UserWarning: egw(t) is nonmonotonic in the range [np.float64(0.11180092537581351), np.float64(0.23710541508380023)]
For more verbose output use `debug_level=1`.
  debug_message(message, self.debug_level,
/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:178

Unable to determine eccentricity. Skipping


ICTSEccParallel12
M = 48.0
Unable to determine eccentricity. Skipping


ICTSEccParallel17
M = 54.0
measured eccentricity on first try


ICTSEccParallel13
M = 47.0
Unable to determine eccentricity. Skipping


ICTSEccParallel14
M = 55.0
measured eccentricity on first try


ICTSEccParallel15
M = 48.0
Unable to determine eccentricity. Skipping


ICTSEccParallel16
M = 47.0
measured eccentricity with fref_in = 21.0




/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1788: UserWarning: egw(t) is nonmonotonic in the range [np.float64(0.10331346736863918), np.float64(0.20269559076828614)]
For more verbose output use `debug_level=1`.
  debug_message(message, self.debug_level,


In [21]:
unsuccessful_sims

['ICTSEccParallel05',
 'ICTSEccParallel10',
 'ICTSEccParallel12',
 'ICTSEccParallel13',
 'ICTSEccParallel15']

#### ICTSEccParallel12, 13, 15

In [22]:
mode = [2, 2]
dataDict = {}
iota = np.pi/6
DL = 410

for sim in ['ICTSEccParallel12', 'ICTSEccParallel13', 'ICTSEccParallel15']:
    print(sim)
    M = inj_vals_df['mtotal'][sim]
    print('M =', M)
    dataDict[sim] = {}
    wf_files_path = os.path.join(sim_files_dir, 
                                 f'ICTSEccParallel_{sims_at_highest_lev_df["levs"][sim]}_data_{mode[0]}{mode[1]}_mode.h5')
    with h5py.File(wf_files_path, 'r') as data_from_file:
        approx_data = data_from_file[sim]
        t_by_M = approx_data['t_by_M'][:]
        Re_rh_by_M = approx_data['Real_rh_by_M'][:]
        Im_rh_by_M = approx_data['Imag_rh_by_M'][:]
    
        t_sec = t_by_M*M*MTSUN_SI
        t_interp = np.arange(t_by_M[0], t_by_M[-1], 0.1)
        t_interp_sec = t_interp*M*MTSUN_SI  #Multiply by total mass in seconds
    
        Y_lm = waveform_modes.get_glm(mode[0], mode[1], iota)  #taking phi_0 = 0
        hp = Re_rh_by_M * M_by_R(M, DL) * Y_lm
        hc = -Im_rh_by_M * M_by_R(M, DL) * Y_lm
    
        hp_interp = interpolate(t_interp_sec, t_sec, hp)
        hc_interp = interpolate(t_interp_sec, t_sec, hc)
        hlm_interp = hp_interp - 1j * hc_interp
    
        dataDict[sim]['t'] = t_interp_sec
        dataDict[sim]['hlm'] = {(2,2):hlm_interp}
    
        try:
            ecc_dict = measure_eccentricity(fref_in=20, dataDict=dataDict[sim], method='Frequency', 
                                            extra_kwargs={'treat_mid_points_between_pericenters_as_apocenters': True})
            print('measured eccentricity on first try')
            inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
            inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']
        except Exception as e:
            try:                
                if 'Reference frequency is outside the allowed range' in str(e):
                    fref_in = np.ceil(float(str(e).split('[')[-1].split(']')[0].split(',')[0]))
                    ecc_dict = measure_eccentricity(fref_in=fref_in, dataDict=dataDict[sim], method='Frequency', 
                                                    extra_kwargs={'treat_mid_points_between_pericenters_as_apocenters': True})
                    print(f'measured eccentricity with fref_in = {fref_in}')
                else:
                    raise Exception
                inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
                inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']
            except:
                unsuccessful_sims.append(sim)
                print('Unable to determine eccentricity. Skipping')
    unsuccessful_sims.remove(sim)

ICTSEccParallel12
M = 48.0
measured eccentricity on first try
ICTSEccParallel13
M = 47.0


/home/divyajyoti/miniconda3/envs/gw_eccentricity/lib/python3.10/site-packages/gw_eccentricity/eccDefinition.py:1788: UserWarning: egw(t) is nonmonotonic in the range [np.float64(0.10465425876801097), np.float64(0.18174311758954004)]
For more verbose output use `debug_level=1`.
  debug_message(message, self.debug_level,


measured eccentricity on first try
ICTSEccParallel15
M = 48.0
measured eccentricity with fref_in = 21.0


In [23]:
unsuccessful_sims

['ICTSEccParallel05', 'ICTSEccParallel10']

In [24]:
inj_vals_df

,mchirp,chi_eff,chi_p,mtotal,q,metadata_ecc,gwecc_ecc,gwecc_mean_anomaly
sim_names,,,,,,,,
EccContPrecDiff001,13.1857,0.0253,0.3000,36.0,3.0,0.04,NaN,NaN
EccContPrecDiff003,35.5784,-0.0676,0.2236,102.0,3.5,0.10,NaN,NaN
EccContPrecDiff004,38.6868,0.3474,0.2952,89.0,1.1,0.15,NaN,NaN
EccContPrecDiff005,43.9628,0.0046,0.4999,101.0,1.0,0.18,NaN,NaN
EccContPrecDiff006,49.2702,0.2033,0.4217,116.0,1.5,0.21,NaN,NaN
EccContPrecDiff007,27.7469,0.0025,0.4123,72.0,2.5,0.08,NaN,NaN
EccContPrecDiff008,17.4110,0.0013,0.5000,40.0,1.0,0.18,NaN,NaN
EccPrecDiff002,13.0583,0.1997,0.2005,30.0,1.0,0.10,NaN,NaN
ICTSEccParallel01,19.7786,0.0005,0.3000,54.0,3.0,0.04,NaN,NaN


#### ICTSEccParallel05

In [228]:
mode = [2, 2]
dataDict = {}
iota = np.pi/6
DL = 410
unsuccessful_sims = []

sim = 'ICTSEccParallel05'

M = inj_vals_df['mtotal'][sim]
print('M =', M)
dataDict[sim] = {}
wf_files_path = os.path.join(sim_files_dir, 
                             f'ICTSEccParallel_{sims_at_highest_lev_df["levs"][sim]}_data_{mode[0]}{mode[1]}_mode.h5')
with h5py.File(wf_files_path, 'r') as data_from_file:
    approx_data = data_from_file[sim]
    t_by_M = approx_data['t_by_M'][:]
    Re_rh_by_M = approx_data['Real_rh_by_M'][:]
    Im_rh_by_M = approx_data['Imag_rh_by_M'][:]

    t_sec = t_by_M*M*MTSUN_SI
    t_interp = np.arange(t_by_M[0], t_by_M[-1], gwe_defaults["deltaTOverM"])
    t_interp_sec = t_interp*M*MTSUN_SI  #Multiply by total mass in seconds

    Y_lm = waveform_modes.get_glm(mode[0], mode[1], iota)  #taking phi_0 = 0
    hp = Re_rh_by_M * M_by_R(M, DL) * Y_lm
    hc = -Im_rh_by_M * M_by_R(M, DL) * Y_lm

    hp_interp = interpolate(t_interp_sec, t_sec, hp)
    hc_interp = interpolate(t_interp_sec, t_sec, hc)
    hlm_interp = hp_interp - 1j * hc_interp

    dataDict[sim]['t'] = t_interp_sec
    dataDict[sim]['hlm'] = {(2,2):hlm_interp}

    ecc_dict = measure_eccentricity(fref_in=20, dataDict=dataDict[sim], method='AmplitudeFits')
    inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
    inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']

M = 110.0


Exception: Number of pericenters found = 0.
Can not build frequency interpolant through the pericenters.


#### ICTSEccParallel10

In [233]:
mode = [2, 2]
dataDict = {}
iota = np.pi/6
DL = 410
unsuccessful_sims = []

sim = 'ICTSEccParallel10'

M = inj_vals_df['mtotal'][sim]
print('M =', M)
dataDict[sim] = {}
wf_files_path = os.path.join(sim_files_dir, 
                             f'ICTSEccParallel_{sims_at_highest_lev_df["levs"][sim]}_data_{mode[0]}{mode[1]}_mode.h5')
with h5py.File(wf_files_path, 'r') as data_from_file:
    approx_data = data_from_file[sim]
    t_by_M = approx_data['t_by_M'][:]
    Re_rh_by_M = approx_data['Real_rh_by_M'][:]
    Im_rh_by_M = approx_data['Imag_rh_by_M'][:]

    t_sec = t_by_M*M*MTSUN_SI
    t_interp = np.arange(t_by_M[0], t_by_M[-1], 0.01)
    t_interp_sec = t_interp*M*MTSUN_SI  #Multiply by total mass in seconds

    Y_lm = waveform_modes.get_glm(mode[0], mode[1], iota)  #taking phi_0 = 0
    hp = Re_rh_by_M * M_by_R(M, DL) * Y_lm
    hc = -Im_rh_by_M * M_by_R(M, DL) * Y_lm

    hp_interp = interpolate(t_interp_sec, t_sec, hp)
    hc_interp = interpolate(t_interp_sec, t_sec, hc)
    hlm_interp = hp_interp - 1j * hc_interp

    dataDict[sim]['t'] = t_interp_sec
    dataDict[sim]['hlm'] = {(2,2):hlm_interp}

    ecc_dict = measure_eccentricity(fref_in=22, dataDict=dataDict[sim], method='AmplitudeFits',
                                    extra_kwargs={'treat_mid_points_between_pericenters_as_apocenters':True})
    inj_vals_df.loc[sim, 'gwecc_ecc'] = ecc_dict['eccentricity']
    inj_vals_df.loc[sim, 'gwecc_mean_anomaly'] = ecc_dict['mean_anomaly']

M = 49.0


Exception: omega22 averaged [pericenter to pericenter] are non-monotonic.
First non-monotonicity occurs at peak number 0, where omega22 drops from 133.52602810034597 to 133.23946638619157, a decrease by 0.2865617141544021.
Total number of places of non-monotonicity is 1.
Last one occurs at peak number 0.
For more verbose output use `debug_level=1` and for diagnostic plot use `debug_plots=True` in extra_kwargs
Possible fixes: 
   - Increase sampling rate of data
   - Add to extra_kwargs the option 'treat_mid_points_between_pericenters_as_apocenters': True